### wps_climdex_gsl

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) [climdex.gsl](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=10) function

Computes the growing season length (GSL): Growing season length is the number of days between the startof the first spell of warm days in the first half of the year, defined as six or more days with mean temperature above 5 degrees Celsius, and the start of the first spell of cold days in the second half of the year, defined as six or more days with a mean temperature below 5 degrees Celsius

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.output_handling import rda_to_vector, auto_construct_outputs
from wps_tools.testing import get_target_url
from quail.utils import test_rda_output

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_gsl?

Signature:
quail.climdex_gsl(
    climdex_input=None,
    loglevel='INFO',
    output_file='output.rda',
    vector_name='output_vector',
    gsl_mode='GSL',
    output_formats=None,
)
Docstring:
Computes the growing season length (GSL)

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename to store the output Rdata (extension .rda)
vector_name : string
    Name to label the output vector
gsl_mode : {'GSL', 'GSL_first', 'GSL_max', 'GSL_sum'}string
    Growing season length method to use. The three alternate modes provided ('GSL_first', 'GSL_max', and 'GSL_sum') are for testing purposes only.
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/code/birds/quail/</tmp/quail-venv/lib/p

#### Run wps_climdex_gsl Process with rda input

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="gsl_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_gsl(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            output_file=output_file.name,
        )
gsl_url = output.get()[0]

#### Run wps_climdex_gsl Process with rds input

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="gsl_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_gsl(
            climdex_input=resource_filename("tests","data/climdexInput.rds"),
            output_file=output_file.name,
        )
gsl_url_rds = output.get()[0]

#### Run wps_climdex_gsl Process with Multiple input

In [8]:
climdex_inputs = [
    resource_filename("tests","data/climdexInput.rds"),
    resource_filename("tests","data/climdexInput.rda")
]
with NamedTemporaryFile(suffix=".rda", prefix="gsl_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_gsl(
            climdex_input=climdex_inputs,
            output_file=output_file.name,
        )
gsl_url_rds = output.get()[0]

Access the output with **rda_to_vector()** or **auto_construct_outputs()** from **wps_tools.output_handling**

In [9]:
gsl = rda_to_vector(gsl_url, "gsl0_ci")
# use print() to see whole vector
print(f"Growing season length\n{gsl}")

Growing season length
1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 
  NA  344   NA   NA  330   NA  283  315  320  339  309  310  317  269  355  324 
1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 
 287   NA   NA  305  330  282  361  270  329   NA   NA  336   NA  338  341  349 
1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 
  NA  349   NA   NA  330  315  360  339  362  319  359   NA  322   NA 



In [10]:
auto_construct_outputs(output.get())

[R object with classes: ('numeric',) mapped to:
 [     nan, 344.000000,      nan,      nan, ..., 359.000000,      nan, 322.000000,      nan],
 R object with classes: ('numeric',) mapped to:
 [     nan, 344.000000,      nan,      nan, ..., 359.000000,      nan, 322.000000,      nan]]

#### Test output against expected output

In [11]:
test_rda_output(
    gsl_url, "gsl0_ci", "expected_gsl.rda", "expected_gsl_vector"
)